In [21]:
ATOMPATH = "../../Atoms.jl"
push!(LOAD_PATH, ATOMPATH)
using ASE

In [81]:
function monkhorstpackgrid(cell, nkpoints)
    # TODO HUAJIE
	kx, ky, kz = nkpoints
	if kx == 0 && ky == 0 && kz == 0
		K = [0.;0.;0.]
		weight = 1.0
	elseif kx == 0 && ky == 0 && kz != 0
		K = zeros(kz, 3)
		weight = zeros(kz)
		v1 = cell[1,:][:]
    	v2 = cell[2,:][:]
    	v3 = cell[3,:][:]
    	c12 = cross(v1,v2)
    	b3 = 2 * π * c12 / dot(v3,c12)
    	k_step = b3 / (kz+1)
    	w_step = norm(b3) / (kz+1)
    	for k = 1:kz
        	K[k,:] = -b3/2 + k * k_step
        	weight[k] = w_step
    	end
	else
		error("This boundary condition has not been implemented yet!")
	end
	return K, weight
end


monkhorstpackgrid (generic function with 1 method)

In [131]:
at = bulk("Al")
atm = repeat(at, (5, 5, 1))
print(atm)
nlist = ASENeighborList(atm, 2.0)
print(nlist)

ASE.ASEAtoms(PyObject Atoms(symbols='Al25', positions=..., cell=[[0.0, 10.125, 10.125], [10.125, 0.0, 10.125], [2.025, 2.025, 0.0]], pbc=[True, True, True]))ASE.ASENeighborList(PyObject <ase.calculators.neighborlist.NeighborList instance at 0x4514e18>,[2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0])

In [130]:
nlist

ASE.ASENeighborList(PyObject <ase.calculators.neighborlist.NeighborList instance at 0x427f128>,[4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0  …  4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0])

In [122]:
K, weight = monkhorstpackgrid(cell(atm), nk)

(
4x3 Array{Float64,2}:
 0.0       0.0       -0.0     
 0.258567  0.258567  -0.258567
 0.517135  0.517135  -0.517135
 0.775702  0.775702  -0.775702,

[1.4478516948726465,2.895703389745293,2.895703389745293,1.4478516948726465])

In [110]:
function monkhorstpackgrid(cell, nkpoints)
    # TODO HUAJIE
	kx, ky, kz = nkpoints
	if kx != 0 || ky != 0
		error("This boundary condition has not been implemented yet!")
	end
	if kz != 0 && mod(kz,2) == 0
		error("It is better to choos k as an odd number so that Γ point is included!")
	end
	if kz == 0
		K = [0.;0.;0.]
		weight = 1.0
	end
	if kz != 0 && mod(kz,2) != 0
		# allocate space, note that symmetry of k-space can be applied
		nk = Int((kz+1)/2)
		K = zeros(nk, 3)
		weight = zeros(nk)
		# compute the lattice vector of reciprocal space
		v1 = cell[1,:][:]
    	v2 = cell[2,:][:]
    	v3 = cell[3,:][:]
    	c12 = cross(v1,v2)
    	b3 = 2 * π * c12 / dot(v3,c12)
		# evaluate K and weight
    	k_step = b3 / (kz+1)
    	w_step = norm(b3) / (kz+1)
    	for k = 1:nk
        	K[k,:] = (k-1) * k_step
        	weight[k] = w_step * 2.0  ## CHECK
    	end
	else
		error("This boundary condition has not been implemented yet!")
	end
	return K, weight
end


monkhorstpackgrid (generic function with 1 method)

In [111]:
a=3.0

3.0

In [105]:
Int(a)

3

In [114]:

# HUAJIE ------------------------------------------------------------------------
function monkhorstpackgrid(cell, nkpoints)
	kx, ky, kz = nkpoints
	if kx != 0 || ky != 0
		error("This boundary condition has not been implemented yet!")
	end
	# open boundarycondition
	if kz == 0 || kz == 1
		K = [0.;0.;0.]
		weight = 1.0
	else
		if mod(kz,2) == 1
			error("k should be an even number in Monkhorst-Pack grid!")
		end
   		# compute the lattice vector of reciprocal space
		v1 = cell[1,:][:]
    	v2 = cell[2,:][:]
    	v3 = cell[3,:][:]
    	c12 = cross(v1,v2)
    	b3 = 2 * π * c12 / dot(v3,c12)
		# K = {b/kz * j + shift}_{j=-kz/2+1,...,kz/2} with shift = 0.0
		# so we can exploit the symmetry of the brillouin zone 
		nk = Int(kz/2) + 1
		K = zeros(nk, 3)
		weight = zeros(nk)
		k_step = b3 / kz
    	w_step = norm(b3) / kz+1
    	for k = 1:nk
        	K[k,:] = (k-1) * k_step
			if k == 1 || k == nk
	        	weight[k] = w_step
			else 
	        	weight[k] = w_step * 2.0 
			end
    	end
	end
	return K, weight
end


monkhorstpackgrid (generic function with 1 method)